In [82]:
%%file esn_classifier.py
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import scipy.sparse as sparse

class EsnClassifier(BaseEstimator, ClassifierMixin):
    """Echo state network classifier"""
    def __init__(self, \
                 density=1, reservoirSize=100, outputleakingRate=1, \
                 inputSize=1, outputSize=1, leakingRate = 1, \
                 randomState=None, regularizationCoefficient=10e-6,
                 alpha=None):
        """
        Called when initializing the classifier
        """
        self.density = density
        self.reservoirSize = reservoirSize
        self.leakingRate = leakingRate
        self.randomState = randomState
        self.outputSize = outputSize
        self.inputSize = inputSize
        self.alpha = alpha
        self.regularizationCoefficient = regularizationCoefficient
    
    def fit(self, X, y=None):
        """
        """
        # FIXME: add Asserts or try/catch
        
        examples, sequenceLength = X.shape
        self.Win_, self.W_ = self.build_reservoirs()
    
        bias = np.ones((1, examples))
        
        # run the reservoir with the data and collect X
        x = np.zeros((self.reservoirSize,examples))
        for pic in range(sequenceLength):
            u = X[:, pic]
            x = (1-self.leakingRate)*x + self.leakingRate*np.tanh( np.dot( self.Win_, np.vstack((bias,u)) ) + np.dot( self.W_, x ) )
            print(pic, end="\r")
        
        # Reservoir values
        self.X = np.vstack((bias,x))
        self.y = y
        # Fit linear regression
        self.refit(self.regularizationCoefficient)
        return self
    
    def refit(self, regularizationCoefficient):
        """
        Fit regression with parameter regularizationCoefficient
        """
        self.Wout_ = np.dot( np.dot(self.y.T,self.X.T), np.linalg.inv( np.dot(self.X,self.X.T) + \
            regularizationCoefficient*np.eye(1+self.reservoirSize) ) ) 
        return self
    
    def predict(self, X, y=None):
        '''
        '''
        examples, sequenceLength = X.shape
        x = np.zeros((self.reservoirSize,examples))
        bias = np.ones((1, examples))
        for pix in range(sequenceLength):
            u = X[:, pix]
            x = (1-self.leakingRate)*x + self.leakingRate*np.tanh( np.dot( self.Win_, np.vstack((bias,u)) ) + np.dot( self.W_, x ) )
            print(pix, end="\r")
            
        y = np.dot( self.Wout_, np.vstack((bias,x)) ).T 
        return np.array(np.argmax(y, axis=1))
    
    
    # Helpers to build reservoir
    def __spectral_radius(self, matrix):
        '''
        Calculate spectral radius of matrix. 
        Spectral radius is max absolute eigenvalue.
        '''
        # FIXME: remove code below
        inner = matrix
        eigenvalues = np.linalg.eig(inner)[0]
        return max(abs(eigenvalues))
    
    def build_reservoirs(self):
        '''
        Generate reservoirs
        '''
        # FIXME: move to spartial
        
        # include bias term
        if self.alpha is None:
            Win =  sparse.rand(self.reservoirSize, self.inputSize + 1, density=self.density, random_state=self.randomState)
            Win -= (Win.sign()*0.5)
            Win = Win.toarray()
        else:
            Win = np.ones((self.reservoirSize, self.inputSize + 1)) * self.alpha

        W = sparse.rand(self.reservoirSize, self.reservoirSize, density=self.density, random_state=self.randomState)
        W -= W.sign()*0.5
        W *= 1.25/self.__spectral_radius(W.toarray())
        return (Win, W.toarray())

Overwriting esn_classifier.py


In [83]:
from esn_classifier import EsnClassifier
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.sparse as sparse

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from scipy.io import loadmat

import math
import time


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
my_data = np.genfromtxt('train.cv', delimiter=',')

def extract(row):
    num = row[0]
    pic = row[1:]
    return (num, pic)

def encode2vec(num):
    converted = int(num) % 10
    result = np.zeros((10,), dtype='int')
    result[converted] = 1
    return result

def normalize_pixel(pixels):
    return pixels/256

Y = np.ndarray((42000,1), dtype='int')
Y_t = np.ndarray((42000,10), dtype='int')
U = np.ndarray((42000,784))

for i in range(1,42001):
    num, pic = extract(my_data[i, :])
    Y[i-1, :] = num
    U[i-1, :] = normalize_pixel(pic)
    
X_train, X_test, y_train, y_test = train_test_split(U, Y, test_size=0.2, random_state=42, stratify=Y)
y_v_train = np.ndarray((y_train.shape[0], 10), dtype=int, buffer=np.array([encode2vec(y) for y in y_train]))
y_v_test = np.ndarray((y_test.shape[0], 10), dtype=int, buffer=np.array([encode2vec(y) for y in y_test]))

In [70]:
def process(n, alpha, regulizations):
    print("Processing reservouir size %d for a %f. " %(n, alpha), end="")
    clf = EsnClassifier(reservoirSize=n, 
                        outputSize=10,
                        regularizationCoefficient=regulizations[0],
                        alpha=alpha)
    start = time.time()
    clf.fit(X_train, y_v_train)
    endFit = time.time()
    time2fit = endFit - start
    predicted = clf.predict(X_test)
    endPredict = time.time()
    time2predict = endPredict - endFit 
    acc = { regulizations[0]: metrics.accuracy_score(y_test, predicted)}
    for r in regulizations[1:]:
        clf.refit(r)
        predicted = clf.predict(X_test)
        acc[r] = metrics.accuracy_score(y_test, predicted)
    print("Accuracy: %f%%"% round(max(acc.values()) * 100))
    return (acc, (time2fit, time2predict))

resultsAccuracy = {}
resultsTime = {}
for n in [10, 25, 50, 100, 250]:
    resultsAccuracy[n] = {}
    resultsTime[n] = {}
    for a in range(1,10):
        alpha = a / 10.0
        resultsAccuracy[n][a] = {}
        resultsTime[n][a] = {}
        acc, t = process(n, alpha, [10e-4, 10e-6])
        resultsAccuracy[n][a] = acc
        resultsTime[n][a] = t

Accuracy: 12.000000%r size 10 for a 0.100000. 0
Accuracy: 18.000000%r size 10 for a 0.200000. 0
Accuracy: 12.000000%r size 10 for a 0.300000. 0
Accuracy: 12.000000%r size 10 for a 0.400000. 0
Accuracy: 12.000000%r size 10 for a 0.500000. 0
Accuracy: 18.000000%r size 10 for a 0.600000. 0
Accuracy: 12.000000%r size 10 for a 0.700000. 0
Accuracy: 12.000000%r size 10 for a 0.800000. 0
Accuracy: 12.000000%r size 10 for a 0.900000. 0
Accuracy: 20.000000%r size 25 for a 0.100000. 0
Accuracy: 13.000000%r size 25 for a 0.200000. 0
Accuracy: 27.000000%r size 25 for a 0.300000. 0
Accuracy: 18.000000%r size 25 for a 0.400000. 0
Accuracy: 23.000000%r size 25 for a 0.500000. 0
Accuracy: 12.000000%r size 25 for a 0.600000. 0
Accuracy: 12.000000%r size 25 for a 0.700000. 0
Accuracy: 12.000000%r size 25 for a 0.800000. 0
Accuracy: 12.000000%r size 25 for a 0.900000. 0
Accuracy: 35.000000%r size 50 for a 0.100000. 0
Accuracy: 15.000000%r size 50 for a 0.200000. 0
Accuracy: 25.000000%r size 50 for a 0.30

In [78]:
process(250, 0.2, [10e-4])

Accuracy: 18.000000%r size 250 for a 0.200000. 0


({0.001: 0.1819047619047619}, (515.816437959671, 134.00792598724365))

In [84]:
clf = EsnClassifier(reservoirSize=250, 
                        outputSize=10,
                        regularizationCoefficient=10e4,
                        alpha=0.2)

In [85]:
clf.fit(X_train, y_v_train)

EsnClassifier(alpha=0.2, density=1, inputSize=1, leakingRate=1, outputSize=10,
       outputleakingRate=None, randomState=None,
       regularizationCoefficient=100000.0, reservoirSize=250)

In [106]:
yVal = clf.y.argmax(axis=1)

In [107]:
clf.X.shape[0]

251

In [123]:
centers = np.zeros((10, 251))
centersCount = np.zeros((10, 251))

In [124]:
for r in range(clf.X.shape[1]):
    row = clf.X[:, r]
    classs = yVal[r]
    centers[classs, :] = centers[classs, :] + row
    centersCount[classs] += 1

In [125]:
np.divide(centers, centersCount)

array([[ 1.        ,  0.33855048,  0.51380293, ...,  0.63666359,
        -0.75522815,  0.34212203],
       [ 1.        ,  0.33891353,  0.5136968 , ...,  0.63655749,
        -0.7552672 ,  0.34201603],
       [ 1.        ,  0.33925679,  0.51362768, ...,  0.63664059,
        -0.75530024,  0.34190409],
       ..., 
       [ 1.        ,  0.33512948,  0.51308757, ...,  0.63992878,
        -0.7524992 ,  0.34394097],
       [ 1.        ,  0.33876968,  0.51341616, ...,  0.63721199,
        -0.75526177,  0.34220059],
       [ 1.        ,  0.34003165,  0.51362339, ...,  0.63671153,
        -0.75509789,  0.34196515]])

In [127]:
centerOfMasses = centerOfMass(clf.X, clf.y)

In [197]:
t = centerOfMasses[0] - centerOfMasses[1]
np.dot(t.T, t)

0.0060281828052843583

In [145]:
avgDistance(centerOfMasses)

2.0932008533152846

In [199]:
%%time
t = variance(clf.X, clf.y)
print(t)

1.1573242238
CPU times: user 951 ms, sys: 17 ms, total: 968 ms
Wall time: 968 ms


In [181]:
np.sqrt(np.dot(temp.T, temp))

0.64875304019948166

In [182]:
centersCount[exampleClass][0]

312318.0

0.011008496340842272

In [412]:
A = np.array([[1,50,2,1.4],[2,51,3,1.2]])
b = np.array([[1, 0],[0, 1], [1, 0], [1,0]])
_, classesSize = b.shape
centers, classCount = centerOfMass(A, b, classesSize)
centersOfMasses = np.divide(centers, classCount)
dist = avgDistance(centersOfMasses, classesSize)
var = variance(A, b, centers, classCount, centersOfMasses, classesSize)

In [413]:
separation(A, b)

24.568632472745261

In [350]:
centersOfMasses

array([[  1.,   2.],
       [ 50.,  51.]])

In [298]:
b.argmax(axis=1)

array([0, 2])

In [331]:
np.all(classCount[0], r < 0)

TypeError: an integer is required

In [324]:
centers

array([[  1.,  50.],
       [  0.,   0.],
       [  2.,  51.]])